In [ ]:
import subprocess
import re
import os
import threading
import json
from bs4 import BeautifulSoup as bs

In [ ]:
cars_id_lst = []
lock_cars_id_lst = threading.Lock()
lock_out_cars_dict = threading.Lock()
lock_out_cars_id_lst = threading.Lock()
lock_out_logs = threading.Lock()
lock_print = threading.Lock()

with open('cars.txt', 'r') as file_cars_id_lst:
    cars_id_lst = [int(i) for i in file_cars_id_lst.readlines()]

In [ ]:
def getFileFromUrl(url):
    try:
        return subprocess.check_output(f'curl {url}', shell=True), 0
    except subprocess.CalledProcessError as grepexc:                                                                                                   
        return None, grepexc.returncode
    
def downloadFileFromUrl(url, outfile):
    try:
        return subprocess.call(f'curl -C -o {outfile} {url}'.format(url), shell=True)
    except subprocess.CalledProcessError as grepexc:                                                                                                   
        return grepexc.returncode
    
def decodeEmail(e):
    de = ''
    k = int(e[:2], 16)
    for i in range(2, len(e)-1, 2):
        de += chr(int(e[i:i+2], 16)^k)

    return de

def fetchCarInfo(idx, save_imgs=True, outdir='./'):
    car_dict = {'id': idx}
    car_html = None
    car_html, returncode = getFileFromUrl(f'https://shobiddak.com/cars/{idx}')
    err_coount = 1
    while returncode != 0 or err_coount> 10:
        err_coount += 1
        car_html, returncode = getFileFromUrl(f'https://shobiddak.com/cars/{idx}')
        
    if returncode != 0:
        return car_dict, returncode
    
    car_bs = bs(car_html)
    if car_bs.find('div', {'class': 'post-price'}) is None:
        return car_dict, -1
    
    car_dict['موديل'] = car_bs.find('h1').get_text()
    car_dict['سنة'] = car_bs.find('h3').get_text().split(' ')[2]
    car_dict['سعر'] = car_bs.find('div', {'class': 'post-price'}).get_text().split('\n')[0].split(' ')[0]
    for kv in car_bs.find('table', {'class': 'list_ads'}).find_all('tr', {'class': 'list-row'}):
        k = kv.find_all()[0].get_text()
        v = re.compile(r' $').sub('', kv.find_all()[1].get_text())
        if k == 'إضافات':
            v = re.compile(r' +').sub(' ', v)
            v = re.compile(r'(\n )+').sub('\n', v).split('\n')
            v = [i for i in v if len(i)> 0]
            
        car_dict[k] = v

    try:
        v = car_bs.find('th', text='معلومات إضافية')
        v = v.parent.parent.find_all('li')
        car_dict['معلومات إضافية'] = [i.get_text() for i in v]
    except:
        pass
    
    try:
        for kv in car_bs.find('tr', id='advertiser_info').find_next().find_next().find_all('tr'):
            k = kv.find_all()[0].get_text()
            if k == 'Email':
                v = decodeEmail(kv.find_all()[1].find_all()[1]['data-cfemail'])
            else:
                v = re.compile(r' $').sub('', kv.find_all()[1].get_text())
            
            car_dict[k] = v
    except:
        for kv in car_bs.find('td', text='حالة الإعلان').parent.parent.find_all('tr'):
            k = kv.find_all()[0].get_text()
            v = kv.find_all()[1].get_text()
            car_dict[k] = re.compile(r' $').sub('', v)

    car_dict['صور'] = [x.find('a')['href'] for x in car_bs.find_all('p', {'class': 'img-thumbnail'})]
    if save_imgs:
        for img_path in car_dict['صور']:
            try:
                img_dir = img_path.split('/')[5]
                os.makedirs(os.path.join(outdir, f'uploads/picture/car/name/{img_dir}'))
            except:
                pass
            # @TODO: Handle except
            downloadFileFromUrl(f'https://shobiddak.com{img_path}', os.path.join(outdir, img_path[1:]))
            
    return car_dict, 0

def fetchCarsInfo(idx_st, idx_en, save_imgs=True, outdir='./', cars_dict={}):
    for idx in range(idx_st, idx_en):
        print(f'\r[+] [Progress] [{idx}/ {idx_en-1}]', end='')
        if idx in cars_dict.keys():
            continue
        car_dict, returncode = fetchCarInfo(idx, save_imgs=save_imgs, outdir=outdir)
        
        if returncode != 0:
            print(f'\t[-] [Error {returncode}] [Debug] [{idx}]')
        cars_dict[idx] = car_dict
        
    return cars_dict

def fetchCarsInfoThread(
    idx_st,
    idx_en,
    save_imgs=False,
    outdir='./', out_cars_dict='cars.json', out_cars_id_lst='cars.txt', out_logs='logs.txt'):
    
    global cars_id_lst
    for idx in range(idx_st, idx_en):
        with lock_cars_id_lst:
            if idx in cars_id_lst:
                continue
            else:
                with lock_print:
                    print(f'\r[+] [Progress] [{idx}/ {idx_en-1}]', end='')

                cars_id_lst.append(idx)

        car_dict, returncode = fetchCarInfo(idx, save_imgs=save_imgs, outdir=outdir)
        if returncode != 0:
            with lock_out_logs, open(out_logs, 'a') as file_logs:
                file_logs.write(f'[-] [Error {returncode}] [Debug] [{idx}]\n')

        with lock_out_cars_dict, open(out_cars_dict, 'a') as file_cars_dict:
            json.dump(car_dict, file_cars_dict, ensure_ascii=False, indent=1)

        with lock_out_cars_id_lst, open(out_cars_id_lst, 'a') as file_cars_id_lst:
            file_cars_id_lst.write(str(idx)+ '\n')
            
    return 0

In [ ]:
%%time
lst_threads = [threading.Thread(target=fetchCarsInfoThread, args=(640000, 690000, _)) for _ in range(8)]
for thread in lst_threads:
    thread.start()
    
for thread in lst_threads:
    thread.join()